In [ ]:
import json
import copy
import random
import time

#Functions for generating game states
def generate_next_gameloop(current_state, current_round):
    """
    Generates the next game loop in a game based on the current state.

    :param current_state: A JSON object representing the current state of the game.
    :return: A JSON object representing the next state of the game.
    """
    # Deep copy the current state to avoid modifying the original state
    new_state = copy.deepcopy(current_state)
    # Increment the game loop number
    new_state[f"loop_{current_round + 1}"] = copy.deepcopy(current_state[f"loop_{current_round}"])

    # Process each bike and its agents
    for bike in new_state[f"loop_{current_round + 1}"]['bikes'].values():
        # Check if the awdi intersects this bike
        awdi_intersects = new_state[f"loop_{current_round + 1}"]['awdi']['target'] == bike['id']
        bike["position"]["x"] = bike["position"]["x"] + random.randint(-10, 10)
        bike["position"]["y"] = bike["position"]["y"] + random.randint(-10, 10)
        # Process each agent on the bike
        for agent in bike['agents'].values():
            # If the awdi intersects, remove all agents from the bike
            if awdi_intersects:
                agent['energy'] = 0  # Setting energy to 0 as they are eliminated

            # Update agent's energy and check if they should be moved to bike 0
            if agent['energy'] > 0:
                agent['energy'] -= 1  # Decrease energy by 1 for this loop
                if agent['energy'] == 0:
                    # Move to bike 0
                    agent['bikeID'] = '0x0'
            else:
                # Already on bike 0 or moved to bike 0
                agent['bikeID'] = '0x0'

    # Handle agents on bike 0 (this can be expanded with more logic if needed)
    # Currently, it only ensures they are on bike 0
    # update awdi position
    new_state[f"loop_{current_round + 1}"]['awdi']['position']['x'] += random.randint(-10, 10)
    new_state[f"loop_{current_round + 1}"]['awdi']['position']['y'] += random.randint(-10, 10)
    return new_state

def generate_initial_state():
    """
    Generates an initial game state with a random number of bikes and agents.

    :return: A JSON object representing the initial state of the game.
    """
    # Define possible colors for agents
    possible_colours = ["Red", "Green", "Blue", "Yellow", "Orange", "Purple", "Pink", "Brown", "Gray", "White"]

    # Initialize the game state
    initial_state = {
        "loop_0": {
            "bikes": {},
            "lootboxes": {},  # This can be populated with lootbox data if needed
            "awdi": {
                "target": None,  # No target initially
                "position": {"x": random.randint(-300, 300), "y": random.randint(-300, 300)},
            }
        },
    }

    # Randomly determine the number of bikes
    num_bikes = random.randint(6, 10)

    for bike_id in range(1, num_bikes + 1):
        # Each bike will have a unique ID
        bike_key = f"bike_{bike_id}"

        # Randomly determine the number of agents on this bike
        num_agents = random.randint(3, 9)
        bike_agents = {}

        for agent_id in range(1, num_agents + 1):
            # Each agent will have a unique ID and randomly assigned color
            agent_key = f"agent_{bike_id}_{agent_id}"
            agent_colour = random.choice(possible_colours)

            # Define the agent
            bike_agents[agent_key] = {
                "id": agent_key,
                "groupID": random.randint(0, 8),
                "colour": agent_colour,
                "points": 0,
                "energy": 100,  # Initial energy can be adjusted
                # The rest of the agent's attributes are placeholders
                "bikes": {},
                "forces": {},
                "lootboxChoice": None,
                "lootDistribution": [],
            }

        # Add the bike and its agents to the game state
        initial_state["loop_0"]["bikes"][bike_key] = {
            "id": bike_key,
            "position": {"x": random.randint(-300, 300), "y": random.randint(-300, 300)},
            "agents": bike_agents
        }

    # add lootboxes
    num_lootboxes = random.randint(4, 12)
    for lootbox_id in range(1, num_lootboxes + 1):
        lootbox_key = f"lootbox_{lootbox_id}"
        initial_state["loop_0"]["lootboxes"][lootbox_key] = {
            "id": lootbox_key,
            "position": {
                "x": random.randint(-500, 500),
                "y": random.randint(-500, 500)
            },
            "colour": random.choice(possible_colours),
        }
    
    # add awdi
    initial_state["loop_0"]["awdi"]["id"] = "awdi"
    initial_state["loop_0"]["awdi"]["position"] = {
        "x": random.randint(-500, 500),
        "y": random.randint(-500, 500)
    }

    return initial_state


In [ ]:
# Generate the initial state and run the game for LOOPS number of loops
initial_state = generate_initial_state()
LOOPS = 10
for i in range(LOOPS):
    initial_state = generate_next_gameloop(initial_state, i)
with open(f"./visualiser/JSON/test2.json", "w") as f:
    json.dump(initial_state, f, indent=4)

In [3]:
import json
with open("../game_dump.json", "r") as f:
    data = json.load(f)

In [11]:
for id, values in data[0]["bikes"].items():
    print(id, values)

003d23f5-21d4-4020-b531-7bd45a3ec84a {'physical_state': {'position': {'x': 2.4648933753913864, 'y': 44.37418512009924}, 'acceleration': 0.8333333333333334, 'velocity': 0.8333333333333334, 'mass': 6}, 'agent_ids': ['5547e839-182a-47e3-bc09-c9c4f6d26b81', '43091250-05e2-4a71-8a48-b5a42e6b735d', '81806a33-8b65-477b-83b2-6d7f62b05e54', 'a6046693-99c8-48f1-9dd6-85ab3e4ccaee', '408ffe98-2065-4003-8bb5-905e6dffa7e4']}
8646710b-6d98-42ea-901c-65b2310df013 {'physical_state': {'position': {'x': 15.751307647239058, 'y': 72.21107002228453}, 'acceleration': 0.5, 'velocity': 0.5, 'mass': 2}, 'agent_ids': ['cf0b6d21-1f3a-4315-8c8e-721d091c7bb5']}
8d4c2a5f-e25c-45c1-b500-bc71a1017935 {'physical_state': {'position': {'x': 5.440868533186091, 'y': 7.161817473112905}, 'acceleration': 0, 'velocity': 0, 'mass': 1}, 'agent_ids': []}


In [14]:
for id, values in data[0]["agents"].items():
    print(id, values)

408ffe98-2065-4003-8bb5-905e6dffa7e4 {'forces': {'pedal': 1, 'brake': 0, 'turning': {'steer_bike': True, 'steering_force': 0.5570415807827523}}, 'energy_level': 0.9, 'points': 0, 'resource_allocation_params': {'need': 0, 'demand': 0, 'provision': 0, 'appropriation': 0}, 'colour': 'purple', 'location': {'x': 2.4648933753913864, 'y': 44.37418512009924}, 'on_bike': True, 'bike_id': '003d23f5-21d4-4020-b531-7bd45a3ec84a'}
43091250-05e2-4a71-8a48-b5a42e6b735d {'forces': {'pedal': 1, 'brake': 0, 'turning': {'steer_bike': True, 'steering_force': 0.5570415807827523}}, 'energy_level': 0.9, 'points': 0, 'resource_allocation_params': {'need': 0, 'demand': 0, 'provision': 0, 'appropriation': 0}, 'colour': 'blue', 'location': {'x': 2.4648933753913864, 'y': 44.37418512009924}, 'on_bike': True, 'bike_id': '003d23f5-21d4-4020-b531-7bd45a3ec84a'}
5547e839-182a-47e3-bc09-c9c4f6d26b81 {'forces': {'pedal': 1, 'brake': 0, 'turning': {'steer_bike': True, 'steering_force': 0.5570415807827523}}, 'energy_level

In [4]:
for bike in data[0]["bikes"]:
    print(bike["agents"][0])

TypeError: string indices must be integers

In [13]:
print(bike)

{'id': 'c3f60ba6-9917-4af2-854c-dddda481f1a5', 'physical_state': {'position': {'x': 379.2924977174136, 'y': 18.741400417727487}, 'acceleration': 0, 'velocity': 0, 'mass': 50}, 'agents': []}


In [ ]:
# for audi in data[0]["audi":]
print(data[0]["audi"])

In [14]:
print(data[0]["loot_boxes"][0])

{'id': 'ab3ae98d-cc77-49e7-a61a-12f1e192adbd', 'physical_state': {'position': {'x': 174.19206318464458, 'y': 401.2478960094951}, 'acceleration': 0, 'velocity': 0, 'mass': 0}, 'total_resources': 6.916819697775168, 'colour': 'orange'}
